In [ ]:
# 아이디와 비밀번호를 입력해주세요
# 아이디와 비밀번호를 입력한 후 기본적으로 모두 실행을 하면 되지만, 만약 오류가 나왔을 경우 다시 실행해야하는 부분은 따로 표시해두었습니다
id = ''
password = ''

In [ ]:
# COLAP selenium환경 설정
!pip install selenium
!apt-get update
!apt install chromium-chromedriver

from selenium import webdriver
from urllib.request import urlopen
from bs4 import BeautifulSoup as bs
from urllib.parse import quote_plus
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import ActionChains
import pandas as pd
import csv
import time


chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', chrome_options=chrome_options)

     |████████████████████████████████| 958 kB 14.0 MB/s 
     |████████████████████████████████| 138 kB 67.3 MB/s 
     |████████████████████████████████| 356 kB 76.4 MB/s 
     |████████████████████████████████| 55 kB 2.8 MB/s 
     |████████████████████████████████| 3.6 MB 59.6 MB/s 
     |████████████████████████████████| 54 kB 2.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.7 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Get:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:2 http://archive.ubuntu

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: DeprecationWarning: use options instead of chrome_options


In [ ]:
# 만약 밑에서 오류가 날경우 대부분 로그인이 씹혀서 발생한 겁니다 
# 이부분부터 다시 실행 해주시면 가능할껍니다
action = ActionChains(driver)
login_url = 'https://app.deepnatural.ai/Auth/SignIn'
driver.get(login_url)

driver.current_url
time.sleep(2)

input_eamil = driver.find_element(By.NAME, 'email')
input_eamil.send_keys(id)

input_password = driver.find_element(By.NAME, 'password')
input_password.send_keys(password, Keys.ENTER)

In [ ]:
# 사이트가 매일 생성되는 배너의 갯수가 달라집니다.
# 생성되는 배너의 갯수만큼 이 코드를 반복 실행하면 됩니다.
# 'check.png' 이미지 파일이 생성 되는데 해당 이미지 파일에서 어떠한 배너도 보이지 않으면 됩니다

url = 'https://app.deepnatural.ai/projects'
driver.get(url)
time.sleep(2)
driver.current_url

try:
  banner = driver.find_elements(By.TAG_NAME, 'button')
  banner[-2].click()
except:
  print(i)

time.sleep(2)
driver.save_screenshot('check.png')

True

현재 공개된 프로젝트 크롤링

In [ ]:
# 공개
project_all = driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/div/main/div/div[2]/div/div')
items = project_all.find_elements(By.CLASS_NAME, 'project-card.project-item')

reply_list_able = []

for num in range(len(items)):
  project_all = driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/div/main/div/div[2]/div/div')
  items = project_all.find_elements(By.CLASS_NAME, 'project-card.project-item')
  time.sleep(2)

  data_type = items[num].find_element(By.CLASS_NAME, 'category.label').text
  data_title = items[num].find_element(By.CLASS_NAME, 'title').text

  data_content = ''
  data_contents = items[num].find_elements(By.TAG_NAME, 'p')
  for i in data_contents:
    data_content = data_content + '\n' + i.text

  data_collect_rate = items[num].find_element(By.CLASS_NAME, 'value').text
  data_price = items[num].find_element(By.CLASS_NAME, 'credit').text

  data_detail = ''
  action.click(items[num]).perform()
  time.sleep(2)
  detail_content = driver.find_elements(By.XPATH, '//*[@id="project-modal"]/div/div[2]/div[1]/div/div[2]/div/p/*/*')

  for content in detail_content:
    data_detail = data_detail + '\n' + content.text.strip()

  driver.get(url)
  
  reply_list_able.append([data_type, data_title, data_content, data_collect_rate, data_price, data_detail])

data = pd.DataFrame(reply_list_able)
data.columns = ['type', 'title', 'content', 'rate', 'price', 'detail']

data.to_csv('레이블러_공개.csv', encoding='utf-8-sig')

이후 코드는 레이블러 전체 데이터 리스트 가져오는 코드문

In [ ]:
project_all = driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/div/main/div/div[4]/div/div')
items = project_all.find_elements(By.CLASS_NAME, 'project-row.project-item')
reply_list_all = []

# 스크롤 다운
while(True):
  action.move_to_element(items[-1]).perform()
  time.sleep(2)
  next_item = project_all.find_elements(By.CLASS_NAME, 'project-row.project-item')

  if items[-1] == next_item[-1]:
    break
  else:
    items = next_item

for item in items:
  data_type = item.find_element(By.CLASS_NAME, 'category.label').text
  data_title = item.find_element(By.CLASS_NAME, 'title').text
  try:
    data_content = item.find_element(By.TAG_NAME, 'p').text
  except:
    pass
    
  data_collect_rate = item.find_element(By.CLASS_NAME, 'value').text
  data_price = item.find_element(By.CLASS_NAME, 'credit').text
  
  reply_list_all.append([data_type, data_title, data_content, data_collect_rate, data_price])

# CSV파일로 저장
data = pd.DataFrame(reply_list_all)
data.columns = ['type', 'title', 'content', 'rate', 'price']

data.to_csv('레이블러_전체.csv', encoding='utf-8-sig')